# New functions

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" # -1 cpu

In [2]:
import warnings
warnings.filterwarnings("ignore", message=r"Passing", category=FutureWarning)
from nets import unet
from nets.unet_xception import UnetXception
from PIL import Image, ImageFile
import numpy as np
from openslide import OpenSlide
import csv, glob
import datetime
import re
import time
import tensorflow as tf
import keras.backend as K
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = None
from keras.backend.tensorflow_backend import set_session  
config = tf.ConfigProto()  
config.gpu_options.allow_growth = True  
# config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.Session(config=config))  
import cv2
import pandas as pd
from data_loader import TestGenerator
from extract_patches_module_simple import read_wsi_regions, construct_colored_wsi
from crop_image import image_padding, joint_image
import glob
import keras
from pathlib import Path
import math
import matplotlib.pyplot as plt
import openslide

Using TensorFlow backend.
In /root/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /root/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /root/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


## for file folder

In [3]:
file_path = 'test_IHC/'

In [4]:
img_file = []
exts = ['.tiff', '.tif']

for ext in exts:
    img_file += glob.glob(os.path.join(file_path, '*' + ext))

In [5]:
output_path = 'result'
os.makedirs(output_path, exist_ok=True)

## predict 

In [6]:
today = datetime.date.today()
result_path = os.path.join('result/', 'test_{:02d}{:02d}'.format(today.month, today.day))
# result_path = 'datasets/test_1130/'
if not os.path.exists(result_path):
    os.makedirs(result_path)

In [8]:
def get_best_model(model_dir):
    models = os.listdir(model_dir)
    best_file = ''
    best_loss = float('inf')
    for m in models:
        match = re.search( r'val_loss(.*).h5', m, re.M|re.I)
        if match is None:
            continue
        val_loss = float(match.group(1))
        if val_loss < best_loss:
            best_file = m
    return best_file

In [11]:
model_path = 'logs_512/20230605/'

In [12]:
get_best_model(model_path)

'ep146-loss0.064-val_loss0.065.h5'

In [13]:
batch_size = 8

In [14]:
unet = UnetXception(num_classes = 2, drop_rate = 0.8)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [15]:
unet.load_weights(os.path.join(model_path, get_best_model(model_path)))

##tissue_area

In [16]:
def joint_image(patch_image, item_width, col, row):
    target_size = (int(col * item_width), int(row * item_width))
    print(item_width, col, row)
    result_image = Image.new('RGB', target_size, (255, 255, 255))
    for index, s_image in enumerate(patch_image):
        h = int(index/col)
        w = index%col
        result_image.paste(s_image,(w*item_width, h*item_width))
    return result_image

In [17]:
def image_padding(input_image, item_width):
    #iw, ih = input_image.size
    iw = np.size(input_image,1)
    ih = np.size(input_image,0)
    col = int(np.ceil(iw/item_width))
    row = int(np.ceil(ih/item_width))  #网络的输入为正方形，所以item_width=item_height
    target_size = (int(col*item_width), int(row*item_width))
    image = Image.new('RGB', target_size, "white")
    image.paste(input_image, (0, 0))
    return image, col, row


def cut_image(image, col, row, item_width):
    box_list = []
    count = 0
    for j in range(0, row):
        for i in range(0, col):
            count += 1
            box = (i * item_width, j * item_width, (i + 1) * item_width, (j + 1) * item_width)
            box_list.append(box)
    image_list = []
    for box in box_list:
        tmp = image.crop(box)
        if tmp.size!=(item_width, item_width):
            tmp = image_padding(tmp, item_width)[0]
        image_list.append(tmp)
    return image_list

In [18]:
def get_required_wsi(wsi,level_idx, relative_scale):
    wsi_imgs = wsi.read_region((0, 0), level_idx, wsi.level_dimensions[level_idx])
    wsi_imgs = construct_colored_wsi(wsi_imgs, return_array=False)
    if relative_scale!=1:
        wsi_imgs = wsi_imgs.resize((int(wsi_imgs.size[0]//relative_scale), int(wsi_imgs.size[1]//relative_scale)))
    return wsi_imgs

In [19]:
def get_downsample_tissue(file, required_px_size = 8):
    # 8μm == downsample 16 in mpp 0.5μm/pixel
    wsi = OpenSlide(file)
    mpp = float(wsi.properties["openslide.mpp-x"])
    downsample = required_px_size / mpp
    relative_scale = 1
    level_downsamples = np.array(wsi.level_downsamples)
    if downsample < level_downsamples[0]:
        return wsi, 0, 1
    for i in range(1, len(level_downsamples)):
        if downsample < level_downsamples[i] : 
            if level_downsamples[i] - downsample < 1:  # closer to level_downsamples[i]
                return wsi, i, 1
            else:
                relative_scale = downsample / level_downsamples[i-1]
                if relative_scale <= 1.5:
                    relative_scale = 1
                return wsi, i-1, relative_scale

    relative_scale = downsample / level_downsamples[len(level_downsamples) - 1]
    if relative_scale <= 1.5:
        relative_scale = 1
    return wsi, len(level_downsamples)-1, relative_scale

In [20]:
def get_wsi_level(wsi,level_idx):
    level_id=2**(level_idx)
    mpp=float(wsi.properties[openslide.PROPERTY_NAME_MPP_X])
    total_level=level_id* mpp
    return total_level

In [21]:
def tumor_areas(img_file,model_path):
    for i,file in enumerate(img_file):
        print('Start' + file, end = ',')
        start =time.time()
        wsi,level_idx, relative_scale=get_downsample_tissue(file)
        end1 = time.time()
        wsi_imgs=get_required_wsi(wsi,level_idx, relative_scale)
        width, height = wsi_imgs.size
        row = math.ceil(height / 512)
        col = math.ceil(width / 512)
        patches_512 = cut_image(wsi_imgs, col, row, 512)
        patches_512 = [np.asarray(p) for p in patches_512]
        print('read: {:.2f}'.format(end1 - start), end = ', ')
        unet.load_weights(os.path.join(model_path, get_best_model(model_path)))
        image_512_gen = TestGenerator(patches_512, 512, batch_size, is_cls=False)
        masks = unet.predict_generator(image_512_gen, use_multiprocessing=False, workers=4)
        total=get_wsi_level(wsi,level_idx)
        masks = np.argmax(masks, axis=-1)
        result_image_list = []
        area_list=[]
        for i, image in enumerate(patches_512):
            mask_arr=sum(masks[i])
            #area=(sum(mask_arr))*total*total 
            area=(np.sum(mask_arr))*total*total 
            color_mask = np.array([255, 0, 0], dtype='uint8')
            mask_img = np.where(masks[i][..., None], color_mask, np.asarray(image))
            img_new = cv2.addWeighted(np.asarray(image), 0.7, mask_img, 0.3, 0)
            image = Image.fromarray(img_new)
            result_image_list.append(image)
            area_list.append(area)
        Areas=sum(area_list)
        with open('Area_HE.csv', 'a', encoding='utf-8', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([file,Areas])
        result_image = joint_image(result_image_list, 512, col, row)
        result_image = result_image.crop((0,0,width,height))
        w = result_image.size[0]
        h = result_image.size[1]
        if w < 40000 and h<40000: 
            try:  
                result_image.save(os.path.join(result_path, os.path.splitext(os.path.basename(file))[0] + '.jpg'))
                print('save successfully!  Total time: ', time.time() - start)
            except:
                print('save failed 1!  Total time: ', time.time() - start)
        else:
            if w < 60000:
                try:
                    boxes = [(0, 0,w//2, int(h/2)), (int(w/2), 0, w, int(h/2)), (0, int(h/2), int(w/2), h), (int(w/2), int(h/2), w, h)]
                    for i in range(4):
                        result_image.crop(boxes[i]).save(os.path.join(result_path, os.path.splitext(os.path.basename(file))[0] + '_' + str(i+1) +  '.jpg'))
                    print('save successfully crop 4 parts!  Total time: ' , time.time() - start)
                except:
                    print('save failed 2!  Total time: ', time.time() - start)
            else:
                try:
                    boxes = [(0, 0, w//3, h//2), (w//3, 0, w//3*2, h//2), (w//3*2, 0, w, h//2),
                         (0, h//2, w//3, h), (w//3, h//2, w//3*2, h), (w//3*2, h//2, w, h)]
                    for i in range(6):
                        result_image.crop(boxes[i]).save(os.path.join(result_path, os.path.splitext(os.path.basename(file))[0] + '_' + str(i+1) +  '.jpg'))

                    print('save successfully crop 6 parts!  Total time: ' , time.time() - start)
                except:
                    print('save failed 3!  Total time: ', time.time() - start)
    return result_image,Areas

In [22]:
tumor_areas(img_file,model_path)

Starttest_IHC/100597S01X2_IHC.tif,read: 0.00, 512 4 4
save successfully!  Total time:  5.38815712928772
Starttest_IHC/100013S01_HE.tif,read: 0.00, 512 3 2
save successfully!  Total time:  2.782843589782715


(<PIL.Image.Image image mode=RGB size=1502x955>, 40578138.92543965)

In [ ]:
from openslide import OpenSlide, OpenSlideUnsupportedFormatError
import numpy as np
import openslide
a = OpenSlide('./20230718/1000384S01_HE.tif')
a.level_dimensions
b=a.level_dimensions[4]
print(b)
lrHeight = a.level_dimensions[4][1]
lrWidth = a.level_dimensions[4][0]
mmx = a.properties['openslide.mpp-x']
print(lrHeight,lrWidth,mmx)
np.abs((np.array(a.level_downsamples) -16)).argmin()
np.abs((np.array(a.level_downsamples) -16))
a.properties
a.read_region((0, 0), 4, (512,512)).size